In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

In [2]:
import re
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm.notebook import tqdm_notebook


In [3]:
!pip install transformers

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
from transformers import RobertaTokenizer, TFRobertaModel

In [5]:
dataset = pd.read_csv('test_data.csv')

In [6]:
def remove_words(dataset):
    cleaned_tweet=[]
    for text in dataset:
        words=text.split()
        cleaned_words=[]
        for word in words:
            if not word.startswith('@'):
                cleaned_words.append(word);
        cleaned_text = ' '.join(cleaned_words)
        cleaned_tweet.append(cleaned_text)
    return cleaned_tweet

In [7]:
tweet=dataset['tweet']

In [8]:
tweets=remove_words(tweet)

In [9]:
dataset['tweet']=tweets

In [10]:
import nltk
from nltk.corpus import stopwords

In [11]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [13]:
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
import nltk
nltk.download('omw-1.4')
def clean(text):

  cleanr = re.compile('<[^>]*>')
  cleantext = re.sub(cleanr, ' ', text)

  cleantext = re.sub("[-]", " " , cleantext)

  cleantext = re.sub("[^A-Za-z0-9 ]", " " , cleantext)
  cleantext = cleantext.lower()

  words = nltk.tokenize.word_tokenize(cleantext)
  words_new = [i for i in words if i not in stop_words]

  w = [lemmatizer.lemmatize(word) for word in words_new if len(word)>2]

  return ' '.join(w)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [15]:
dataset['tweet'] = tqdm_notebook(dataset['tweet'].apply(clean))

  0%|          | 0/486 [00:00<?, ?it/s]

In [16]:
dataset

,id,tweet,labels
0,1070378532260470789t,study link hpv vaccine historically high infer...,religious side-effect
1,973746711964372993t,death tainted measles vaccine affecting anti p...,side-effect
2,1043031076787040257t,apreciat videoclip http uknvkypp3w treatment c...,side-effect
3,1066338147527741440t,video mmr vaccine increase risk autism african...,side-effect
4,963522018544152576t,oral polio vaccine infecting unvaccinated kid ...,side-effect
...,...,...,...
481,1099937642169405440t,know hard believe another vaccine scare story ...,none
482,1536674508731518985t,seizure day mmr vaccine tale normal child diag...,side-effect
483,1183314494874968064t,kenyan doctor say unicef making woman barren p...,side-effect
484,1327581896243556352t,experience vaccine 60 school giving sugar cube...,pharma


In [17]:
dataset['labels'] = dataset['labels'].str.split()

In [18]:
dataset

,id,tweet,labels
0,1070378532260470789t,study link hpv vaccine historically high infer...,"[religious, side-effect]"
1,973746711964372993t,death tainted measles vaccine affecting anti p...,[side-effect]
2,1043031076787040257t,apreciat videoclip http uknvkypp3w treatment c...,[side-effect]
3,1066338147527741440t,video mmr vaccine increase risk autism african...,[side-effect]
4,963522018544152576t,oral polio vaccine infecting unvaccinated kid ...,[side-effect]
...,...,...,...
481,1099937642169405440t,know hard believe another vaccine scare story ...,[none]
482,1536674508731518985t,seizure day mmr vaccine tale normal child diag...,[side-effect]
483,1183314494874968064t,kenyan doctor say unicef making woman barren p...,[side-effect]
484,1327581896243556352t,experience vaccine 60 school giving sugar cube...,[pharma]


In [19]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [20]:
def tokenize_tweets(text):
    input_ids = []
    attention_masks = []
    token_type_ids = []

    for tweet in text:
        encoded = tokenizer.encode_plus(
            tweet,
            add_special_tokens=True,
            max_length=300,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors='tf'
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        token_type_ids.append(encoded['token_type_ids'])

    return {
        'input_ids': tf.concat(input_ids, axis=0),
        'attention_mask': tf.concat(attention_masks, axis=0),
        'token_type_ids': tf.concat(token_type_ids, axis=0)
    }

In [21]:
train_tokens = tokenize_tweets(dataset['tweet'])

In [22]:
bert_model = TFRobertaModel.from_pretrained('roberta-base')

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [23]:
input_ids = tf.keras.Input(shape=(300,), dtype=tf.int32)
attention_mask = tf.keras.Input(shape=(300,), dtype=tf.int32)
token_type_ids = tf.keras.Input(shape=(300,), dtype=tf.int32)


In [25]:
dataset['labels']

0      [religious, side-effect]
1                 [side-effect]
2                 [side-effect]
3                 [side-effect]
4                 [side-effect]
                 ...           
481                      [none]
482               [side-effect]
483               [side-effect]
484                    [pharma]
485       [pharma, side-effect]
Name: labels, Length: 486, dtype: object

In [28]:
dataset['labels'].isnull().sum()

6

In [30]:
dataset["labels"].dropna()

0      [religious, side-effect]
1                 [side-effect]
2                 [side-effect]
3                 [side-effect]
4                 [side-effect]
                 ...           
481                      [none]
482               [side-effect]
483               [side-effect]
484                    [pharma]
485       [pharma, side-effect]
Name: labels, Length: 480, dtype: object

In [35]:
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform([[] if isinstance(labels, float) else labels for labels in dataset['labels']])

In [36]:
train_labels.shape

(486, 12)

In [37]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout
bert_output = bert_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)[0]
dropout_rate=0.4
weight_decay=0.001
dropout_layer = Dropout(rate=dropout_rate)(bert_output[:, 0, :])
output = tf.keras.layers.Dense(len(mlb.classes_), activation='sigmoid',kernel_regularizer=regularizers.l2(weight_decay))(dropout_layer)

In [38]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [39]:
model = tf.keras.Model(inputs=[input_ids, attention_mask, token_type_ids], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy',metrics=['binary_accuracy'])

In [40]:
train_inputs = [train_tokens['input_ids'], train_tokens['attention_mask'], train_tokens['token_type_ids']]

In [41]:
model.fit(train_inputs, train_labels, batch_size=16, epochs=10)

Epoch 1/10
31/31 [==============================] - 537s 17s/step - loss: 0.3802 - binary_accuracy: 0.8675
Epoch 2/10
31/31 [==============================] - 504s 16s/step - loss: 0.3467 - binary_accuracy: 0.8884
Epoch 3/10
31/31 [==============================] - 501s 16s/step - loss: 0.3311 - binary_accuracy: 0.8937
Epoch 4/10
31/31 [==============================] - 500s 16s/step - loss: 0.2991 - binary_accuracy: 0.9057
Epoch 5/10
31/31 [==============================] - 501s 16s/step - loss: 0.2689 - binary_accuracy: 0.9132
Epoch 6/10
31/31 [==============================] - 497s 16s/step - loss: 0.2409 - binary_accuracy: 0.9210
Epoch 7/10
31/31 [==============================] - 495s 16s/step - loss: 0.2026 - binary_accuracy: 0.9426
Epoch 8/10
31/31 [==============================] - 495s 16s/step - loss: 0.1777 - binary_accuracy: 0.9463
Epoch 9/10
31/31 [==============================] - 493s 16s/step - loss: 0.1508 - binary_accuracy: 0.9561
Epoch 10/10
31/31 [==================

In [42]:
test_data= pd.read_csv('test.csv')

In [43]:
test_data['tweet'] = tqdm_notebook(test_data['tweet'].apply(clean))

  0%|          | 0/486 [00:00<?, ?it/s]

In [44]:
test_tokens = tokenize_tweets(test_data['tweet'])

In [45]:
test_inputs_bert = [test_tokens['input_ids'], test_tokens['attention_mask'], test_tokens['token_type_ids']]

In [ ]:
model.predict(test_inputs_bert)

In [ ]:
data = model.predict(test_inputs_bert)

In [ ]:
data

In [ ]:
data[0]

In [ ]:
threshold = 0.2

binary_data = (data >= threshold).astype(int)

In [ ]:
binary_data[3]

In [ ]:
print(binary_data)

In [ ]:
activated_classes = mlb.inverse_transform(binary_data)

In [ ]:
mlb_classes = mlb.classes_

print(mlb_classes)





In [ ]:
activated_classes

In [ ]:
test_data['labels']=activated_classes

In [ ]:
test_data['labels'] = test_data['labels'].apply(lambda x: ' '.join(map(str, x)))

In [ ]:
test_data

In [ ]:
test_tweet= pd.read_csv('test.csv')

In [ ]:
test_data['tweet']=test_tweet['tweet']

In [ ]:
test_data

In [ ]:
test_data.to_csv('test_data_roberta.csv', index=False)